In [1]:
import os
import sys
sys.path.append('../python')

import pandas as pd
import numpy as np
import yaml
import time
import api


In [2]:
# Create the dataset to embed

chapter_data = pd.read_csv("../../intermediate_data/los_angeles_chapter_data.csv")
article_data = pd.read_csv("../../intermediate_data/los_angeles_article_data.csv")
section_data = pd.read_csv("../../intermediate_data/los_angeles_section_data.csv")

df = section_data.merge(
    article_data, on=['doc_id','chapter_id','article_id'], how='left'
).merge(
    chapter_data, on=['doc_id','chapter_id'], how='left'
).reset_index(drop=True)

df = df.fillna('')

df['text'] = df['chapter_title'] + ' ' + df['article_title'] + ' ' + df['section_title']
df['id'] = df['doc_id'] + '_' + df['section_id']
df['item_type'] = 'section_title_with_headings'

df.head()

,chapter_id,article_id,section_id,doc_id,section_title,article_title,chapter_title,text,id,item_type
0,JD_C1,JD_C1A1,JD_11.00.,lamunicipalcode,SEC. 11.00. PROVISIONS APPLICABLE TO CODE.,ARTICLE 1 GENERAL PROVISIONS,CHAPTER I GENERAL PROVISIONS AND ZONING,CHAPTER I GENERAL PROVISIONS AND ZONING ARTICL...,lamunicipalcode_JD_11.00.,section_title_with_headings
1,JD_C1,JD_C1A1,JD_11.01.,lamunicipalcode,SEC. 11.01. DEFINITIONS AND INTERPRETATION.,ARTICLE 1 GENERAL PROVISIONS,CHAPTER I GENERAL PROVISIONS AND ZONING,CHAPTER I GENERAL PROVISIONS AND ZONING ARTICL...,lamunicipalcode_JD_11.01.,section_title_with_headings
2,JD_C1,JD_C1A1,JD_11.02.,lamunicipalcode,SEC. 11.02. INCONSISTENT PERMITS AND LICENSES.,ARTICLE 1 GENERAL PROVISIONS,CHAPTER I GENERAL PROVISIONS AND ZONING,CHAPTER I GENERAL PROVISIONS AND ZONING ARTICL...,lamunicipalcode_JD_11.02.,section_title_with_headings
3,JD_C1,JD_C1A1,JD_11.03.,lamunicipalcode,SEC. 11.03. POST WAR RENEWAL OF LICENSES OF CR...,ARTICLE 1 GENERAL PROVISIONS,CHAPTER I GENERAL PROVISIONS AND ZONING,CHAPTER I GENERAL PROVISIONS AND ZONING ARTICL...,lamunicipalcode_JD_11.03.,section_title_with_headings
4,JD_C1,JD_C1A1,JD_11.04.,lamunicipalcode,SEC. 11.04. DELINQUENT ACCOUNTS - UNCOLLECTIBL...,ARTICLE 1 GENERAL PROVISIONS,CHAPTER I GENERAL PROVISIONS AND ZONING,CHAPTER I GENERAL PROVISIONS AND ZONING ARTICL...,lamunicipalcode_JD_11.04.,section_title_with_headings


In [3]:
# Embed and upsert the data

api.embed_and_upsert(df, verbose=True)


0... 128... 256... 384... 512... 640... 768... 896... 1024... 1152... 1280... 1408... 1536... 1664... 1792... 1920... 2048... 2176... 2304... 2432... 2560... 2688... 2816... 2944... 3072... 3200... 3328... 3456... 3584... 3712... 3840... 3968... 4096... 4224... 4352... 4480... 4608... 4736... 4864... 4992... 5120... 5248... {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'main': {'vector_count': 5270}},
 'total_vector_count': 5270}


In [5]:
# Run a similarity search for a query

time.sleep(2)

query = "What regulations determine the minimum setback requirement?"

results = api.similarity_search(query)

results

{'matches': [{'id': 'lamunicipalcode_JD_91.6105.',
              'metadata': {'article_id': 'JD_C9A1',
                           'article_title': 'ARTICLE 1 BUILDINGS [BUILDING '
                                            'CODE]',
                           'chapter_id': 'JD_C9',
                           'chapter_title': 'CHAPTER IX BUILDING REGULATIONS',
                           'doc_id': 'lamunicipalcode',
                           'id': 'lamunicipalcode_JD_91.6105.',
                           'item_type': 'section_title_with_headings',
                           'section_id': 'JD_91.6105.',
                           'section_title': 'SEC. 91.6105. SEPARATION FROM OIL '
                                            'WELLS.',
                           'text': 'CHAPTER IX BUILDING REGULATIONS ARTICLE 1 '
                                   'BUILDINGS [BUILDING CODE] SEC. 91.6105. '
                                   'SEPARATION FROM OIL WELLS.'},
              'score': 0.4799588